In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
lines = pd.read_csv("line.csv", delimiter=",")
nov_recordings = pd.read_csv("SET-nov-2013.csv", delimiter=",", header=None)
dec_recordings = pd.read_csv("SET-dec-2013.csv", delimiter=",", header=None)
import json
f = open("trentino-grid.geojson", 'r')
grid_data = json.loads(f.read())
f.close()

In [3]:
grid_data["features"][0]

{'geometry': {'coordinates': [[[10.914622377957983, 45.682007076150505],
    [10.927456267537572, 45.68179119797432],
    [10.927147329501077, 45.672795442796335],
    [10.914315493899755, 45.67301125363092],
    [10.914622377957983, 45.682007076150505]]],
  'type': 'Polygon'},
 'id': 0,
 'properties': {'cellId': 38},
 'type': 'Feature'}

In [4]:
grid_data["crs"]

{'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'}

In [5]:
class JsonWrapper:
    def __init__(self, json_data):
        self._json = self.__prepare(json_data)
        self._ids = []
        for feature in self._json:
            self._ids.append(feature["id"])
        self._cell_ids = []
        for feature in self._json:
            self._cell_ids.append(feature["properties"]["cellId"])
        
        
    @staticmethod
    def __prepare(json_data):
        return json_data["features"]
        
    def get_polygon(self, cell_id=None):
        if cell_id is None:
            return None
        for feature in self._json:
            if feature["properties"]["cellId"] == cell_id:
                return (feature["id"], feature["geometry"]["coordinates"][0])
        return None
    
    def get_ids(self):
        return self._ids
    
    def get_cell_ids(self):
        return self._cell_ids
    
    def get_data_by_id(self, id=None):
        if id is not None:
            for feature in self._json:
                if feature["id"] == id:
                    return feature
        return None

In [49]:
recordings = pd.concat([dec_recordings, nov_recordings])

In [50]:
dec_recordings.head()

,0,1,2
0,DG1000420,2013-12-01 00:00,36.719997
1,DG1000420,2013-12-01 00:10,36.180000
2,DG1000420,2013-12-01 00:20,35.639999
3,DG1000420,2013-12-01 00:30,34.559998
4,DG1000420,2013-12-01 00:40,34.559998


In [51]:
recordings.head()

,0,1,2
0,DG1000420,2013-12-01 00:00,36.719997
1,DG1000420,2013-12-01 00:10,36.180000
2,DG1000420,2013-12-01 00:20,35.639999
3,DG1000420,2013-12-01 00:30,34.559998
4,DG1000420,2013-12-01 00:40,34.559998


In [9]:
wrapper = JsonWrapper(grid_data)

In [10]:
recordings.to_csv("test.csv")

In [11]:
recordings.columns

Int64Index([0, 1, 2], dtype='int64')

In [12]:
lines.head()

,SQUAREID,LINESET,NR_UBICAZIONI
0,155,DG1031221,4
1,272,DG1031221,72
2,273,DG1031221,21
3,276,DG1003117,34
4,277,DG1003117,3


In [13]:
grid_data["features"][2]

{'geometry': {'coordinates': [[[10.940290010697588, 45.68157387892596],
    [10.953123606466104, 45.68135511904866],
    [10.952810560729896, 45.67235949990072],
    [10.939979018768243, 45.67257819153854],
    [10.940290010697588, 45.68157387892596]]],
  'type': 'Polygon'},
 'id': 2,
 'properties': {'cellId': 40},
 'type': 'Feature'}

In [14]:
grid_data["features"][0]["geometry"]["coordinates"][0]

[[10.914622377957983, 45.682007076150505],
 [10.927456267537572, 45.68179119797432],
 [10.927147329501077, 45.672795442796335],
 [10.914315493899755, 45.67301125363092],
 [10.914622377957983, 45.682007076150505]]

In [15]:
tableau_like_data = pd.DataFrame(columns=['State', 'Point Order', 'PolygonID', 'Latitude', 'Longitude'])

In [17]:
cells = wrapper.get_cell_ids()

In [45]:
data = []
for cell_id in cells:
    point_order = 0
    poly_id, poly = wrapper.get_polygon(cell_id)
    for lat_long in poly:
        data.append([cell_id, point_order, poly_id, lat_long[1], lat_long[0]])
        point_order += 1

In [56]:
tableau_like_data = pd.DataFrame(columns=['Cell ID', 'Point Order', 'PolygonID', 'Latitude', 'Longitude'], data=data[:])

In [57]:
tableau_like_data.head()

,Cell ID,Point Order,PolygonID,Latitude,Longitude
0,38,0,0,45.682007,10.914622
1,38,1,0,45.681791,10.927456
2,38,2,0,45.672795,10.927147
3,38,3,0,45.673011,10.914315
4,38,4,0,45.682007,10.914622


In [58]:
tableau_like_data.to_csv("tableau_custom_poly_full.csv", sep=";")

'Cell ID'

In [63]:
lines.to_csv("lines.csv", sep=";")

In [62]:
tableau_like_data.join(lines, on=tableau_like_data.columns[0])

,Cell ID,Point Order,PolygonID,Latitude,Longitude,SQUAREID,LINESET,NR_UBICAZIONI
0,38,0,0,45.682007,10.914622,978.0,DG1003117,4.0
1,38,1,0,45.681791,10.927456,978.0,DG1003117,4.0
2,38,2,0,45.672795,10.927147,978.0,DG1003117,4.0
3,38,3,0,45.673011,10.914315,978.0,DG1003117,4.0
4,38,4,0,45.682007,10.914622,978.0,DG1003117,4.0
5,39,0,1,45.681791,10.927456,978.0,DG1031221,2.0
6,39,1,1,45.681574,10.940290,978.0,DG1031221,2.0
7,39,2,1,45.672578,10.939979,978.0,DG1031221,2.0
8,39,3,1,45.672795,10.927147,978.0,DG1031221,2.0
9,39,4,1,45.681791,10.927456,978.0,DG1031221,2.0
